<a href="https://colab.research.google.com/github/ajw1587/Pytorch_Study/blob/main/07_MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install torch
!pip install torchvision
! [ ! -z "$COLAB_GPU" ] && pip install torch scikit-learn==0.20.* skorch

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [60]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', cache = False)

print(type(mnist))
mnist.data.shape
mnist.target.shape

# train data와 test data로 split 하기
split_ratio = 0.9
n_train = int(mnist.data.shape[0] * split_ratio)
print(n_train)

n_test = mnist.data.shape[0] - n_train
print(n_test)

fetch_x_train = mnist.data[:n_train]
fetch_y_train = mnist.target[:n_train]
print(fetch_x_train.shape, fetch_y_train.shape)

fetch_x_test = mnist.data[n_train:]
fetch_y_test = mnist.target[n_train:]
print(fetch_x_test.shape, fetch_y_test.shape)

<class 'sklearn.utils.Bunch'>
63000
7000
(63000, 784) (63000,)
(7000, 784) (7000,)


In [61]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(type(x_train))
print(x_train.shape)
print(type(y_train))
print(y_train.shape)

<class 'numpy.ndarray'>
(60000, 28, 28)
<class 'numpy.ndarray'>
(60000,)


In [62]:
mnist_train = dsets.MNIST(root = '../data/', 
                            train=True, 
                            transform = transforms.ToTensor(),
                            download=True)
mnist_test = dsets.MNIST(root = '../data/',
                           train=False,
                           transform = transforms.ToTensor(),
                           download=True)
print(type(mnist_train))
print(mnist_train.data.shape)

print(type(mnist_test))
print(mnist_test.data.shape)

<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([60000, 28, 28])
<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([10000, 28, 28])


In [63]:
data_loader = torch.utils.data.DataLoader(mnist_train,
                                          100,
                                          shuffle = True,
                                          drop_last = True)



In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

linear = torch.nn.Linear(784, 10, bias = True).to(device)

training_epochs = 15
batch_size = 100

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for X, Y in data_loader:
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)
    
    optimizer.zero_grad()
    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost/total_batch
  
  print("Epoch: ", "%04d" % (epoch + 1), "cost = ", "{:.9f}".format(avg_cost))

cuda
Epoch:  0001 cost =  0.535702229
Epoch:  0002 cost =  0.359250218
Epoch:  0003 cost =  0.331297338
Epoch:  0004 cost =  0.316572905
Epoch:  0005 cost =  0.307181686
Epoch:  0006 cost =  0.300416172
Epoch:  0007 cost =  0.295124888
Epoch:  0008 cost =  0.290936857
Epoch:  0009 cost =  0.287408888
Epoch:  0010 cost =  0.284644514
Epoch:  0011 cost =  0.282127410
Epoch:  0012 cost =  0.279899687
Epoch:  0013 cost =  0.277747273
Epoch:  0014 cost =  0.275899172
Epoch:  0015 cost =  0.274502784


In [66]:
# 테스트 데이터를 사용해서 모델을 테스트한다.
with torch.no_grad(): #torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
  X_test = mnist_test.test_data.view(-1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = linear(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item())

  # MNIST 테스트 데이터에서 무작위로 하나 뽑아서 예측
  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = linear(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item())

  # 이건 작동을 안하네...?
  # RuntimeError: CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`
  # 위와 같은 오류가 발생한다...

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


RuntimeError: ignored